In [1]:
# Importation des modules

import pandas as pd
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

INFO - 22:46:06: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 22:46:06: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [2]:
df0 = pd.read_csv('data/data_cleaned_NLP.csv', sep = ',', encoding = 'latin-1')

In [3]:
df1 = df0[['Réplique', 'Groupe', 'tokenized_replique']].copy()

df2 = df1[['Groupe', 'tokenized_replique']].copy()

df_novice = df2[df2['Groupe'] == 'Novice'].copy()
df_exp = df2[df2['Groupe'] == 'Exp'].copy()

In [10]:
df_novice = df_novice.dropna()

df_exp = df_exp.dropna()

In [11]:
sent_novice = [row.split() for row in df_novice['tokenized_replique']]

sent_exp = [row.split() for row in df_exp['tokenized_replique']]

In [12]:
phrases_novice = Phrases(sent_novice, min_count = 3, progress_per = 20000)

phrases_exp = Phrases(sent_exp, min_count = 3, progress_per = 20000)


# min_count : Ignore all words and bigrams with total collected count lower than this value.

INFO - 13:01:05: collecting all words and their counts
INFO - 13:01:05: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 13:01:07: PROGRESS: at sentence #20000, processed 544399 words and 367942 word types
INFO - 13:01:08: PROGRESS: at sentence #40000, processed 1061844 words and 645502 word types
INFO - 13:01:08: collected 773338 word types from a corpus of 1326670 words (unigram + bigrams) and 50191 sentences
INFO - 13:01:08: using 773338 counts as vocab in Phrases<0 vocab, min_count=3, threshold=10.0, max_vocab_size=40000000>
INFO - 13:01:08: collecting all words and their counts
INFO - 13:01:08: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 13:01:09: PROGRESS: at sentence #20000, processed 518787 words and 365570 word types
INFO - 13:01:11: PROGRESS: at sentence #40000, processed 1056472 words and 658662 word types
INFO - 13:01:12: PROGRESS: at sentence #60000, processed 1530225 words and 884119 word types
INFO - 13:01:13: PROGRESS: at sente

In [13]:
bigram_novice = Phraser(phrases_novice)

bigram_exp = Phraser(phrases_exp)

INFO - 13:01:35: source_vocab length 773338
INFO - 13:01:44: Phraser built with 16435 phrasegrams
INFO - 13:01:44: source_vocab length 3813404
INFO - 13:01:55: Phraser added 50000 phrasegrams
INFO - 13:02:34: Phraser built with 73757 phrasegrams


In [14]:
sentences_novice = bigram_novice[sent_novice]

sentences_exp = bigram_exp[sent_exp]

In [15]:
word_freq_novice = defaultdict(int)
word_freq_exp = defaultdict(int)

for sent in sentences_novice:
    for i in sent:
        word_freq_novice[i] += 1
        
for sent in sentences_exp:
    for i in sent:
        word_freq_exp[i] += 1


In [16]:
cores = multiprocessing.cpu_count()

### Choix des hyperparamètres

In [68]:
# Choix de la fenêtre, on aimerait que le modèle aprenne vite, on alongera ensuite le nombre d'epochs

liste_modeles = [Word2Vec(
                     window = i,
                     size = 300,
                     sample = 6e-5, 
                     alpha = 0.03, 
                     min_alpha = 0.0007, 
                     negative = 20,
                     workers = cores - 1,
                     compute_loss = True)
                 
                 for i in range(1, 10)
                ]

tests_fenetre = []

for i in range(len(liste_modeles)):
    
    model = liste_modeles[i]
    model.build_vocab(sentences_novice, progress_per = 10000)


    model.train(sentences_novice, total_examples = model.corpus_count, epochs = 5, report_delay = 1)

    tests_fenetre.append([
            (model.wv.most_similar(positive=['droite'])[i][0],
             model.wv.most_similar(positive=['vitesse'])[i][0],
             model.wv.most_similar(positive=['donc'])[i][0],
            )
         for i in range(10)
         ])

INFO - 00:14:18: collecting all words and their counts
INFO - 00:14:18: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:14:19: PROGRESS: at sentence #10000, processed 233965 words, keeping 30761 word types
INFO - 00:14:20: PROGRESS: at sentence #20000, processed 450032 words, keeping 39850 word types
INFO - 00:14:21: PROGRESS: at sentence #30000, processed 677772 words, keeping 48410 word types
INFO - 00:14:22: PROGRESS: at sentence #40000, processed 880381 words, keeping 51772 word types
INFO - 00:14:22: PROGRESS: at sentence #50000, processed 1095690 words, keeping 55266 word types
INFO - 00:14:22: collected 55316 word types from a corpus of 1099932 raw words and 50191 sentences
INFO - 00:14:22: Loading a fresh vocabulary
INFO - 00:14:25: effective_min_count=5 retains 26248 unique words (47% of original 55316, drops 29068)
INFO - 00:14:25: effective_min_count=5 leaves 1041812 word corpus (94% of original 1099932, drops 58120)
INFO - 00:14:25: deleting the 

INFO - 00:15:24: EPOCH 1 - PROGRESS: at 80.86% examples, 116687 words/s, in_qsize 0, out_qsize 0
INFO - 00:15:25: EPOCH 1 - PROGRESS: at 97.94% examples, 118390 words/s, in_qsize 2, out_qsize 1
INFO - 00:15:25: worker thread finished; awaiting finish of 2 more threads
INFO - 00:15:26: worker thread finished; awaiting finish of 1 more threads
INFO - 00:15:26: worker thread finished; awaiting finish of 0 more threads
INFO - 00:15:26: EPOCH - 1 : training on 1099932 raw words (749440 effective words) took 6.3s, 119339 effective words/s
INFO - 00:15:27: EPOCH 2 - PROGRESS: at 13.68% examples, 111637 words/s, in_qsize 0, out_qsize 1
INFO - 00:15:28: EPOCH 2 - PROGRESS: at 30.72% examples, 116361 words/s, in_qsize 0, out_qsize 0
INFO - 00:15:29: EPOCH 2 - PROGRESS: at 44.53% examples, 113176 words/s, in_qsize 0, out_qsize 0
INFO - 00:15:30: EPOCH 2 - PROGRESS: at 59.03% examples, 110460 words/s, in_qsize 0, out_qsize 0
INFO - 00:15:31: EPOCH 2 - PROGRESS: at 72.66% examples, 105978 words/s, 

INFO - 00:16:22: EPOCH 3 - PROGRESS: at 89.34% examples, 109034 words/s, in_qsize 0, out_qsize 0
INFO - 00:16:23: worker thread finished; awaiting finish of 2 more threads
INFO - 00:16:23: worker thread finished; awaiting finish of 1 more threads
INFO - 00:16:23: worker thread finished; awaiting finish of 0 more threads
INFO - 00:16:23: EPOCH - 3 : training on 1099932 raw words (749271 effective words) took 6.7s, 111363 effective words/s
INFO - 00:16:24: EPOCH 4 - PROGRESS: at 14.59% examples, 121419 words/s, in_qsize 0, out_qsize 0
INFO - 00:16:25: EPOCH 4 - PROGRESS: at 32.44% examples, 121836 words/s, in_qsize 0, out_qsize 0
INFO - 00:16:26: EPOCH 4 - PROGRESS: at 47.15% examples, 118365 words/s, in_qsize 0, out_qsize 0
INFO - 00:16:27: EPOCH 4 - PROGRESS: at 62.67% examples, 116044 words/s, in_qsize 0, out_qsize 0
INFO - 00:16:28: EPOCH 4 - PROGRESS: at 79.48% examples, 115878 words/s, in_qsize 0, out_qsize 0
INFO - 00:16:29: EPOCH 4 - PROGRESS: at 95.64% examples, 115767 words/s, 

INFO - 00:17:19: worker thread finished; awaiting finish of 1 more threads
INFO - 00:17:19: EPOCH 5 - PROGRESS: at 100.00% examples, 121729 words/s, in_qsize 0, out_qsize 1
INFO - 00:17:19: worker thread finished; awaiting finish of 0 more threads
INFO - 00:17:19: EPOCH - 5 : training on 1099932 raw words (749375 effective words) took 6.2s, 121700 effective words/s
INFO - 00:17:19: training on a 5499660 raw words (3746064 effective words) took 32.1s, 116596 effective words/s
INFO - 00:17:19: precomputing L2-norms of word weight vectors
INFO - 00:17:19: collecting all words and their counts
INFO - 00:17:19: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:17:20: PROGRESS: at sentence #10000, processed 233965 words, keeping 30761 word types
INFO - 00:17:21: PROGRESS: at sentence #20000, processed 450032 words, keeping 39850 word types
INFO - 00:17:22: PROGRESS: at sentence #30000, processed 677772 words, keeping 48410 word types
INFO - 00:17:22: PROGRESS: at se

INFO - 00:18:08: estimated required memory for 26248 words and 300 dimensions: 76119200 bytes
INFO - 00:18:08: resetting layer weights
INFO - 00:18:15: training model with 3 workers on 26248 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=6
INFO - 00:18:16: EPOCH 1 - PROGRESS: at 11.29% examples, 91755 words/s, in_qsize 0, out_qsize 0
INFO - 00:18:17: EPOCH 1 - PROGRESS: at 28.29% examples, 105464 words/s, in_qsize 1, out_qsize 0
INFO - 00:18:18: EPOCH 1 - PROGRESS: at 42.19% examples, 106024 words/s, in_qsize 0, out_qsize 0
INFO - 00:18:19: EPOCH 1 - PROGRESS: at 57.44% examples, 106489 words/s, in_qsize 0, out_qsize 0
INFO - 00:18:20: EPOCH 1 - PROGRESS: at 70.25% examples, 103471 words/s, in_qsize 0, out_qsize 0
INFO - 00:18:21: EPOCH 1 - PROGRESS: at 87.49% examples, 106164 words/s, in_qsize 0, out_qsize 0
INFO - 00:18:22: worker thread finished; awaiting finish of 2 more threads
INFO - 00:18:22: worker thread finished; awaiting finish of 1 more threads

INFO - 00:19:14: EPOCH 2 - PROGRESS: at 100.00% examples, 101542 words/s, in_qsize 0, out_qsize 1
INFO - 00:19:14: worker thread finished; awaiting finish of 0 more threads
INFO - 00:19:14: EPOCH - 2 : training on 1099932 raw words (748969 effective words) took 7.4s, 101523 effective words/s
INFO - 00:19:15: EPOCH 3 - PROGRESS: at 14.59% examples, 119437 words/s, in_qsize 0, out_qsize 0
INFO - 00:19:16: EPOCH 3 - PROGRESS: at 32.44% examples, 122039 words/s, in_qsize 0, out_qsize 0
INFO - 00:19:17: EPOCH 3 - PROGRESS: at 46.07% examples, 116082 words/s, in_qsize 0, out_qsize 0
INFO - 00:19:18: EPOCH 3 - PROGRESS: at 60.80% examples, 114078 words/s, in_qsize 0, out_qsize 0
INFO - 00:19:19: EPOCH 3 - PROGRESS: at 76.24% examples, 112640 words/s, in_qsize 1, out_qsize 0
INFO - 00:19:20: EPOCH 3 - PROGRESS: at 91.94% examples, 112384 words/s, in_qsize 0, out_qsize 0
INFO - 00:19:21: worker thread finished; awaiting finish of 2 more threads
INFO - 00:19:21: worker thread finished; awaiting 

INFO - 00:20:13: worker thread finished; awaiting finish of 0 more threads
INFO - 00:20:13: EPOCH - 4 : training on 1099932 raw words (749412 effective words) took 6.9s, 108704 effective words/s
INFO - 00:20:14: EPOCH 5 - PROGRESS: at 12.88% examples, 107218 words/s, in_qsize 0, out_qsize 0
INFO - 00:20:15: EPOCH 5 - PROGRESS: at 29.08% examples, 109962 words/s, in_qsize 0, out_qsize 0
INFO - 00:20:17: EPOCH 5 - PROGRESS: at 44.53% examples, 112383 words/s, in_qsize 0, out_qsize 0
INFO - 00:20:18: EPOCH 5 - PROGRESS: at 60.80% examples, 113896 words/s, in_qsize 0, out_qsize 0
INFO - 00:20:19: EPOCH 5 - PROGRESS: at 75.18% examples, 111247 words/s, in_qsize 0, out_qsize 0
INFO - 00:20:20: EPOCH 5 - PROGRESS: at 87.49% examples, 106795 words/s, in_qsize 0, out_qsize 0
INFO - 00:20:20: worker thread finished; awaiting finish of 2 more threads
INFO - 00:20:20: worker thread finished; awaiting finish of 1 more threads
INFO - 00:20:20: worker thread finished; awaiting finish of 0 more thread

In [72]:
tests_fenetre

[[('résolu', 'suivis', 'donne_avis'),
  ('vécu', 'multiplient', 'adopté'),
  ('assemblée_sénat', 'gratuitement', 'voter'),
  ('représentés', 'conflit_intérêts', 'amendements'),
  ('abouti', 'faire_croire', 'déposé'),
  ('voter_texte', 'allégements', 'émets_donc'),
  ('débouché', 'vingt_sept', 'sous_amendements'),
  ('fait_unanimité', 'donnent', 'voilà_pourquoi'),
  ('france_comores', 'instaurée', 'gouvernement'),
  ('accord_trouvé', 'réclament', 'adopter')],
 [('pouvions', 'saurait_être', 'conséquent'),
  ('dépassé', 'reconstruction', 'article'),
  ('gauche', 'multiplient', 'donne_avis'),
  ('avril', 'rendra', 'voter'),
  ('vécu', 'bonne_nouvelle', 'rédaction'),
  ('résolu', 'confort', 'adopté'),
  ('représentés', 'regroupement', 'donc_satisfait'),
  ('faisait', 'cherché', 'texte'),
  ('attend', 'ajoutent', 'déposé'),
  ('référendum', 'continueront', 'adopter')],
 [('gauche', 'plus_élevés', 'sous_amendements'),
  ('pouvions', 'rendra', 'donne_avis'),
  ('résolu', 'massivement', 'rédact

On choisit window = 4 qualitativement (on se restreint d'abord aux modèles qui associent en premier 'gauche' à 'droite', puis on sélectionne sur la pertinence des autres mots sélectionnés)

In [80]:
# Maintenant qu'on a la fenêtre, on fait le choix du nombre d'époques lors de l'apprentissage. 
# On reprend le même procédé pour tester la fiabilité

tests_epochs = []

liste_nb_epochs = [1, 10, 15, 30]

for nb_epochs in liste_nb_epochs:
    
    model = Word2Vec(
                     window = 4,
                     size = 300,
                     sample = 6e-5, 
                     alpha = 0.03, 
                     min_alpha = 0.0007, 
                     negative = 20,
                     workers = cores - 1,
                     compute_loss = True)
    
    model.build_vocab(sentences_novice, progress_per = 10000)

    model.train(sentences_novice, total_examples = model.corpus_count, epochs = nb_epochs, report_delay = 1)

    tests_epochs.append([
            (model.wv.most_similar(positive=['droite'])[i][0],
             model.wv.most_similar(positive=['vitesse'])[i][0],
             model.wv.most_similar(positive=['donc'])[i][0],
            )
         for i in range(10)
         ])

INFO - 01:04:04: collecting all words and their counts
INFO - 01:04:04: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 01:04:05: PROGRESS: at sentence #10000, processed 233965 words, keeping 30761 word types
INFO - 01:04:06: PROGRESS: at sentence #20000, processed 450032 words, keeping 39850 word types
INFO - 01:04:07: PROGRESS: at sentence #30000, processed 677772 words, keeping 48410 word types
INFO - 01:04:08: PROGRESS: at sentence #40000, processed 880381 words, keeping 51772 word types
INFO - 01:04:09: PROGRESS: at sentence #50000, processed 1095690 words, keeping 55266 word types
INFO - 01:04:09: collected 55316 word types from a corpus of 1099932 raw words and 50191 sentences
INFO - 01:04:09: Loading a fresh vocabulary
INFO - 01:04:09: effective_min_count=5 retains 26248 unique words (47% of original 55316, drops 29068)
INFO - 01:04:09: effective_min_count=5 leaves 1041812 word corpus (94% of original 1099932, drops 58120)
INFO - 01:04:09: deleting the 

INFO - 01:05:11: EPOCH 7 - PROGRESS: at 57.44% examples, 105880 words/s, in_qsize 0, out_qsize 0
INFO - 01:05:12: EPOCH 7 - PROGRESS: at 71.59% examples, 104363 words/s, in_qsize 0, out_qsize 0
INFO - 01:05:13: EPOCH 7 - PROGRESS: at 84.93% examples, 101802 words/s, in_qsize 1, out_qsize 0
INFO - 01:05:14: EPOCH 7 - PROGRESS: at 97.79% examples, 100627 words/s, in_qsize 4, out_qsize 0
INFO - 01:05:14: worker thread finished; awaiting finish of 2 more threads
INFO - 01:05:14: worker thread finished; awaiting finish of 1 more threads
INFO - 01:05:14: worker thread finished; awaiting finish of 0 more threads
INFO - 01:05:14: EPOCH - 7 : training on 1099932 raw words (748935 effective words) took 7.4s, 101529 effective words/s
INFO - 01:05:15: EPOCH 8 - PROGRESS: at 9.69% examples, 80663 words/s, in_qsize 1, out_qsize 0
INFO - 01:05:16: EPOCH 8 - PROGRESS: at 26.72% examples, 100547 words/s, in_qsize 0, out_qsize 0
INFO - 01:05:17: EPOCH 8 - PROGRESS: at 38.94% examples, 97063 words/s, in_

INFO - 01:06:18: EPOCH 14 - PROGRESS: at 81.71% examples, 97302 words/s, in_qsize 0, out_qsize 0
INFO - 01:06:19: worker thread finished; awaiting finish of 2 more threads
INFO - 01:06:19: worker thread finished; awaiting finish of 1 more threads
INFO - 01:06:19: EPOCH 14 - PROGRESS: at 100.00% examples, 102236 words/s, in_qsize 0, out_qsize 1
INFO - 01:06:19: worker thread finished; awaiting finish of 0 more threads
INFO - 01:06:19: EPOCH - 14 : training on 1099932 raw words (749076 effective words) took 7.3s, 102217 effective words/s
INFO - 01:06:20: EPOCH 15 - PROGRESS: at 12.09% examples, 101778 words/s, in_qsize 0, out_qsize 0
INFO - 01:06:21: EPOCH 15 - PROGRESS: at 23.37% examples, 91117 words/s, in_qsize 0, out_qsize 0
INFO - 01:06:22: EPOCH 15 - PROGRESS: at 38.12% examples, 96759 words/s, in_qsize 0, out_qsize 0
INFO - 01:06:23: EPOCH 15 - PROGRESS: at 53.64% examples, 102741 words/s, in_qsize 0, out_qsize 0
INFO - 01:06:24: EPOCH 15 - PROGRESS: at 65.34% examples, 98374 word

INFO - 01:07:20: EPOCH 22 - PROGRESS: at 94.60% examples, 114803 words/s, in_qsize 0, out_qsize 0
INFO - 01:07:21: worker thread finished; awaiting finish of 2 more threads
INFO - 01:07:21: worker thread finished; awaiting finish of 1 more threads
INFO - 01:07:21: worker thread finished; awaiting finish of 0 more threads
INFO - 01:07:21: EPOCH - 22 : training on 1099932 raw words (749082 effective words) took 6.5s, 114410 effective words/s
INFO - 01:07:22: EPOCH 23 - PROGRESS: at 10.44% examples, 89331 words/s, in_qsize 0, out_qsize 0
INFO - 01:07:23: EPOCH 23 - PROGRESS: at 24.42% examples, 94685 words/s, in_qsize 0, out_qsize 0
INFO - 01:07:24: EPOCH 23 - PROGRESS: at 38.12% examples, 95364 words/s, in_qsize 0, out_qsize 0
INFO - 01:07:25: EPOCH 23 - PROGRESS: at 52.74% examples, 98725 words/s, in_qsize 1, out_qsize 0
INFO - 01:07:26: EPOCH 23 - PROGRESS: at 66.47% examples, 98291 words/s, in_qsize 0, out_qsize 0
INFO - 01:07:27: EPOCH 23 - PROGRESS: at 83.31% examples, 100459 words/

INFO - 01:08:19: EPOCH 31 - PROGRESS: at 96.69% examples, 117800 words/s, in_qsize 1, out_qsize 0
INFO - 01:08:20: worker thread finished; awaiting finish of 2 more threads
INFO - 01:08:20: worker thread finished; awaiting finish of 1 more threads
INFO - 01:08:20: worker thread finished; awaiting finish of 0 more threads
INFO - 01:08:20: EPOCH - 31 : training on 1099932 raw words (749424 effective words) took 6.3s, 118680 effective words/s
INFO - 01:08:21: EPOCH 32 - PROGRESS: at 14.59% examples, 116166 words/s, in_qsize 0, out_qsize 0
INFO - 01:08:22: EPOCH 32 - PROGRESS: at 29.08% examples, 108333 words/s, in_qsize 0, out_qsize 0
INFO - 01:08:23: EPOCH 32 - PROGRESS: at 42.19% examples, 104932 words/s, in_qsize 0, out_qsize 0
INFO - 01:08:24: EPOCH 32 - PROGRESS: at 58.21% examples, 107409 words/s, in_qsize 0, out_qsize 0
INFO - 01:08:25: EPOCH 32 - PROGRESS: at 75.18% examples, 110138 words/s, in_qsize 0, out_qsize 0
INFO - 01:08:26: EPOCH 32 - PROGRESS: at 93.76% examples, 113700 w

...

INFO - 01:27:58: EPOCH 209 - PROGRESS: at 85.67% examples, 102649 words/s, in_qsize 0, out_qsize 0
INFO - 01:28:00: EPOCH 209 - PROGRESS: at 97.79% examples, 99912 words/s, in_qsize 0, out_qsize 0
INFO - 01:28:00: worker thread finished; awaiting finish of 2 more threads
INFO - 01:28:00: worker thread finished; awaiting finish of 1 more threads
INFO - 01:28:00: worker thread finished; awaiting finish of 0 more threads
INFO - 01:28:00: EPOCH - 209 : training on 1099932 raw words (749465 effective words) took 7.6s, 98698 effective words/s
INFO - 01:28:01: EPOCH 210 - PROGRESS: at 8.94% examples, 72520 words/s, in_qsize 0, out_qsize 0
INFO - 01:28:02: EPOCH 210 - PROGRESS: at 24.42% examples, 92961 words/s, in_qsize 0, out_qsize 0
INFO - 01:28:03: EPOCH 210 - PROGRESS: at 41.41% examples, 104782 words/s, in_qsize 1, out_qsize 0
INFO - 01:28:04: EPOCH 210 - PROGRESS: at 59.03% examples, 111402 words/s, in_qsize 0, out_qsize 0
INFO - 01:28:05: EPOCH 210 - PROGRESS: at 78.07% examples, 11479

INFO - 01:28:56: worker thread finished; awaiting finish of 2 more threads
INFO - 01:28:56: worker thread finished; awaiting finish of 1 more threads
INFO - 01:28:56: worker thread finished; awaiting finish of 0 more threads
INFO - 01:28:56: EPOCH - 218 : training on 1099932 raw words (749305 effective words) took 5.7s, 132241 effective words/s
INFO - 01:28:57: EPOCH 219 - PROGRESS: at 15.51% examples, 125246 words/s, in_qsize 0, out_qsize 0
INFO - 01:28:58: EPOCH 219 - PROGRESS: at 34.14% examples, 129303 words/s, in_qsize 0, out_qsize 0
INFO - 01:28:59: EPOCH 219 - PROGRESS: at 51.93% examples, 129204 words/s, in_qsize 0, out_qsize 0
INFO - 01:29:00: EPOCH 219 - PROGRESS: at 69.26% examples, 128691 words/s, in_qsize 0, out_qsize 0
INFO - 01:29:01: EPOCH 219 - PROGRESS: at 89.32% examples, 129289 words/s, in_qsize 0, out_qsize 0
INFO - 01:29:01: worker thread finished; awaiting finish of 2 more threads
INFO - 01:29:01: worker thread finished; awaiting finish of 1 more threads
INFO - 0

INFO - 01:29:50: EPOCH 228 - PROGRESS: at 30.72% examples, 116913 words/s, in_qsize 1, out_qsize 0
INFO - 01:29:51: EPOCH 228 - PROGRESS: at 46.07% examples, 117093 words/s, in_qsize 0, out_qsize 0
INFO - 01:29:52: EPOCH 228 - PROGRESS: at 64.46% examples, 121228 words/s, in_qsize 1, out_qsize 0
INFO - 01:29:53: EPOCH 228 - PROGRESS: at 84.14% examples, 122783 words/s, in_qsize 0, out_qsize 0
INFO - 01:29:54: worker thread finished; awaiting finish of 2 more threads
INFO - 01:29:54: worker thread finished; awaiting finish of 1 more threads
INFO - 01:29:54: worker thread finished; awaiting finish of 0 more threads
INFO - 01:29:54: EPOCH - 228 : training on 1099932 raw words (748984 effective words) took 6.1s, 122935 effective words/s
INFO - 01:29:55: EPOCH 229 - PROGRESS: at 15.51% examples, 124992 words/s, in_qsize 0, out_qsize 0
INFO - 01:29:56: EPOCH 229 - PROGRESS: at 33.28% examples, 126840 words/s, in_qsize 0, out_qsize 0
INFO - 01:29:57: EPOCH 229 - PROGRESS: at 50.17% examples, 

INFO - 01:30:49: EPOCH 237 - PROGRESS: at 28.29% examples, 107760 words/s, in_qsize 0, out_qsize 0
INFO - 01:30:50: EPOCH 237 - PROGRESS: at 39.79% examples, 100247 words/s, in_qsize 0, out_qsize 0
INFO - 01:30:51: EPOCH 237 - PROGRESS: at 51.93% examples, 97293 words/s, in_qsize 0, out_qsize 0
INFO - 01:30:53: EPOCH 237 - PROGRESS: at 65.34% examples, 96603 words/s, in_qsize 0, out_qsize 0
INFO - 01:30:54: EPOCH 237 - PROGRESS: at 79.48% examples, 95385 words/s, in_qsize 0, out_qsize 0
INFO - 01:30:55: EPOCH 237 - PROGRESS: at 96.69% examples, 99421 words/s, in_qsize 0, out_qsize 0
INFO - 01:30:55: worker thread finished; awaiting finish of 2 more threads
INFO - 01:30:55: worker thread finished; awaiting finish of 1 more threads
INFO - 01:30:55: worker thread finished; awaiting finish of 0 more threads
INFO - 01:30:55: EPOCH - 237 : training on 1099932 raw words (749251 effective words) took 7.4s, 100699 effective words/s
INFO - 01:30:56: EPOCH 238 - PROGRESS: at 13.68% examples, 1145

INFO - 01:31:50: EPOCH 246 - PROGRESS: at 14.59% examples, 119537 words/s, in_qsize 0, out_qsize 0
INFO - 01:31:51: EPOCH 246 - PROGRESS: at 31.64% examples, 118000 words/s, in_qsize 0, out_qsize 0
INFO - 01:31:52: EPOCH 246 - PROGRESS: at 49.26% examples, 123346 words/s, in_qsize 1, out_qsize 0
INFO - 01:31:53: EPOCH 246 - PROGRESS: at 68.31% examples, 125512 words/s, in_qsize 1, out_qsize 0
INFO - 01:31:54: EPOCH 246 - PROGRESS: at 85.67% examples, 124214 words/s, in_qsize 0, out_qsize 0
INFO - 01:31:55: worker thread finished; awaiting finish of 2 more threads
INFO - 01:31:55: worker thread finished; awaiting finish of 1 more threads
INFO - 01:31:55: worker thread finished; awaiting finish of 0 more threads
INFO - 01:31:55: EPOCH - 246 : training on 1099932 raw words (749007 effective words) took 5.9s, 126006 effective words/s
INFO - 01:31:56: EPOCH 247 - PROGRESS: at 15.51% examples, 125857 words/s, in_qsize 0, out_qsize 0
INFO - 01:31:57: EPOCH 247 - PROGRESS: at 34.14% examples, 

In [81]:
tests_epochs

[[('gauche', 'limitations', 'car'),
  ('contre_gauche', 'carré', 'effet'),
  ('extrême_droite', 'croisière', 'si'),
  ('extrême_gauche', 'rouler', 'aussi'),
  ('mme_louwagie', 'insuline', 'également'),
  ('communistes', 'kilomètres_heure', 'pourquoi'),
  ('droite_hémicycle', 'vitesse_moyenne', 'tout'),
  ('groupuscules', 'engins', 'ailleurs'),
  ('udi_agir', 'francfort', 'non'),
  ('insoumis', 'signalisation', 'bien')]]

### Entrainement des modèles

In [17]:
w2v_model_novice = Word2Vec(
                     window = 4,
                     size = 300,
                     sample = 6e-5, 
                     alpha = 0.03, 
                     min_alpha = 0.0007, 
                     negative = 20,
                     workers = cores - 1,
                     compute_loss = True)

w2v_model_exp = Word2Vec(
                     window = 4,
                     size = 300,
                     sample = 6e-5, 
                     alpha = 0.03, 
                     min_alpha = 0.0007, 
                     negative = 20,
                     workers = cores - 1,
                     compute_loss = True)

In [ ]:
# Modèle novice

t = time()

w2v_model_novice.build_vocab(sentences_novice, progress_per = 10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()

w2v_model_novice.train(sentences_novice, total_examples = w2v_model_novice.corpus_count, epochs = 300, report_delay = 1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [18]:
# Modèle non-novice

t = time()

w2v_model_exp.build_vocab(sentences_exp, progress_per = 20000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()

w2v_model_exp.train(sentences_exp, total_examples = w2v_model_exp.corpus_count, epochs = 300, report_delay = 1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 13:03:16: collecting all words and their counts
INFO - 13:03:16: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:03:18: PROGRESS: at sentence #20000, processed 424284 words, keeping 53196 word types
INFO - 13:03:20: PROGRESS: at sentence #40000, processed 863897 words, keeping 75438 word types
INFO - 13:03:21: PROGRESS: at sentence #60000, processed 1250465 words, keeping 88018 word types
INFO - 13:03:22: PROGRESS: at sentence #80000, processed 1648160 words, keeping 98000 word types
INFO - 13:03:24: PROGRESS: at sentence #100000, processed 2066008 words, keeping 106108 word types
INFO - 13:03:26: PROGRESS: at sentence #120000, processed 2524332 words, keeping 113167 word types
INFO - 13:03:27: PROGRESS: at sentence #140000, processed 2853841 words, keeping 116972 word types
INFO - 13:03:28: PROGRESS: at sentence #160000, processed 3119058 words, keeping 119889 word types
INFO - 13:03:29: PROGRESS: at sentence #180000, processed 3345978 words, keeping

Time to build vocab: 0.96 mins


INFO - 13:04:15: EPOCH 1 - PROGRESS: at 1.32% examples, 95537 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:16: EPOCH 1 - PROGRESS: at 2.64% examples, 89973 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:17: EPOCH 1 - PROGRESS: at 4.40% examples, 96343 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:18: EPOCH 1 - PROGRESS: at 5.92% examples, 97216 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:19: EPOCH 1 - PROGRESS: at 7.38% examples, 97515 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:20: EPOCH 1 - PROGRESS: at 8.72% examples, 95532 words/s, in_qsize 1, out_qsize 0
INFO - 13:04:22: EPOCH 1 - PROGRESS: at 10.42% examples, 95890 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:23: EPOCH 1 - PROGRESS: at 12.01% examples, 95662 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:24: EPOCH 1 - PROGRESS: at 13.66% examples, 96425 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:25: EPOCH 1 - PROGRESS: at 15.69% examples, 98268 words/s, in_qsize 0, out_qsize 0
INFO - 13:04:26: EPOCH 1 - PROGRESS: at 17.12%

INFO - 13:05:41: EPOCH 2 - PROGRESS: at 41.42% examples, 113466 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:42: EPOCH 2 - PROGRESS: at 43.37% examples, 113971 words/s, in_qsize 1, out_qsize 0
INFO - 13:05:43: EPOCH 2 - PROGRESS: at 45.12% examples, 114246 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:44: EPOCH 2 - PROGRESS: at 46.38% examples, 113215 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:45: EPOCH 2 - PROGRESS: at 47.92% examples, 112561 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:46: EPOCH 2 - PROGRESS: at 49.12% examples, 111309 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:47: EPOCH 2 - PROGRESS: at 50.82% examples, 111105 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:48: EPOCH 2 - PROGRESS: at 52.23% examples, 110046 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:49: EPOCH 2 - PROGRESS: at 53.62% examples, 108994 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:50: EPOCH 2 - PROGRESS: at 54.89% examples, 107921 words/s, in_qsize 0, out_qsize 0
INFO - 13:05:51: EPOCH 2 - PRO

INFO - 13:07:05: EPOCH 3 - PROGRESS: at 86.79% examples, 107064 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:07: EPOCH 3 - PROGRESS: at 88.03% examples, 106260 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:08: EPOCH 3 - PROGRESS: at 89.23% examples, 105671 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:09: EPOCH 3 - PROGRESS: at 90.77% examples, 105695 words/s, in_qsize 1, out_qsize 0
INFO - 13:07:10: EPOCH 3 - PROGRESS: at 92.42% examples, 105202 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:11: EPOCH 3 - PROGRESS: at 93.17% examples, 103706 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:12: EPOCH 3 - PROGRESS: at 94.30% examples, 102933 words/s, in_qsize 1, out_qsize 0
INFO - 13:07:13: EPOCH 3 - PROGRESS: at 95.76% examples, 102701 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:14: EPOCH 3 - PROGRESS: at 97.41% examples, 102549 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:15: EPOCH 3 - PROGRESS: at 99.21% examples, 102530 words/s, in_qsize 0, out_qsize 0
INFO - 13:07:15: worker thread

...

INFO - 17:12:55: EPOCH 295 - PROGRESS: at 28.49% examples, 113655 words/s, in_qsize 0, out_qsize 0
INFO - 17:12:56: EPOCH 295 - PROGRESS: at 31.10% examples, 113593 words/s, in_qsize 0, out_qsize 0
INFO - 17:12:57: EPOCH 295 - PROGRESS: at 33.04% examples, 113628 words/s, in_qsize 0, out_qsize 0
INFO - 17:12:58: EPOCH 295 - PROGRESS: at 38.24% examples, 113384 words/s, in_qsize 0, out_qsize 0
INFO - 17:12:59: EPOCH 295 - PROGRESS: at 40.07% examples, 113584 words/s, in_qsize 0, out_qsize 0
INFO - 17:13:00: EPOCH 295 - PROGRESS: at 41.67% examples, 113463 words/s, in_qsize 0, out_qsize 0
INFO - 17:13:01: EPOCH 295 - PROGRESS: at 43.47% examples, 113607 words/s, in_qsize 0, out_qsize 0
INFO - 17:13:02: EPOCH 295 - PROGRESS: at 45.24% examples, 113573 words/s, in_qsize 0, out_qsize 0
INFO - 17:13:03: EPOCH 295 - PROGRESS: at 46.88% examples, 113639 words/s, in_qsize 0, out_qsize 0
INFO - 17:13:04: EPOCH 295 - PROGRESS: at 48.63% examples, 113574 words/s, in_qsize 0, out_qsize 0
INFO - 17:

INFO - 17:14:18: EPOCH 296 - PROGRESS: at 89.23% examples, 113227 words/s, in_qsize 0, out_qsize 0
INFO - 17:14:19: EPOCH 296 - PROGRESS: at 90.86% examples, 113333 words/s, in_qsize 1, out_qsize 0
INFO - 17:14:20: EPOCH 296 - PROGRESS: at 93.18% examples, 113267 words/s, in_qsize 0, out_qsize 0
INFO - 17:14:21: EPOCH 296 - PROGRESS: at 95.08% examples, 113207 words/s, in_qsize 0, out_qsize 0
INFO - 17:14:22: EPOCH 296 - PROGRESS: at 97.06% examples, 113277 words/s, in_qsize 0, out_qsize 0
INFO - 17:14:23: EPOCH 296 - PROGRESS: at 99.08% examples, 113260 words/s, in_qsize 0, out_qsize 0
INFO - 17:14:24: worker thread finished; awaiting finish of 2 more threads
INFO - 17:14:24: worker thread finished; awaiting finish of 1 more threads
INFO - 17:14:24: worker thread finished; awaiting finish of 0 more threads
INFO - 17:14:24: EPOCH - 296 : training on 8521446 raw words (6016368 effective words) took 53.0s, 113438 effective words/s
INFO - 17:14:25: EPOCH 297 - PROGRESS: at 1.41% examples,

INFO - 17:15:39: EPOCH 298 - PROGRESS: at 41.54% examples, 113064 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:40: EPOCH 298 - PROGRESS: at 43.37% examples, 113091 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:41: EPOCH 298 - PROGRESS: at 45.12% examples, 113217 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:42: EPOCH 298 - PROGRESS: at 46.76% examples, 113307 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:43: EPOCH 298 - PROGRESS: at 48.63% examples, 113268 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:44: EPOCH 298 - PROGRESS: at 50.41% examples, 113229 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:45: EPOCH 298 - PROGRESS: at 52.36% examples, 113284 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:46: EPOCH 298 - PROGRESS: at 54.37% examples, 113339 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:47: EPOCH 298 - PROGRESS: at 56.33% examples, 113350 words/s, in_qsize 0, out_qsize 0
INFO - 17:15:48: EPOCH 298 - PROGRESS: at 58.14% examples, 113403 words/s, in_qsize 0, out_qsize 0
INFO - 17:

INFO - 17:17:03: worker thread finished; awaiting finish of 2 more threads
INFO - 17:17:03: worker thread finished; awaiting finish of 1 more threads
INFO - 17:17:03: worker thread finished; awaiting finish of 0 more threads
INFO - 17:17:03: EPOCH - 299 : training on 8521446 raw words (6014976 effective words) took 52.9s, 113652 effective words/s
INFO - 17:17:04: EPOCH 300 - PROGRESS: at 1.41% examples, 106318 words/s, in_qsize 0, out_qsize 0
INFO - 17:17:05: EPOCH 300 - PROGRESS: at 3.28% examples, 112195 words/s, in_qsize 0, out_qsize 0
INFO - 17:17:06: EPOCH 300 - PROGRESS: at 5.05% examples, 112852 words/s, in_qsize 0, out_qsize 0
INFO - 17:17:07: EPOCH 300 - PROGRESS: at 6.80% examples, 113150 words/s, in_qsize 0, out_qsize 0
INFO - 17:17:08: EPOCH 300 - PROGRESS: at 8.50% examples, 113962 words/s, in_qsize 0, out_qsize 0
INFO - 17:17:09: EPOCH 300 - PROGRESS: at 10.57% examples, 113880 words/s, in_qsize 0, out_qsize 0
INFO - 17:17:10: EPOCH 300 - PROGRESS: at 12.41% examples, 114

Time to train the model: 253.69 mins


In [19]:
# Sauvegarde des modèles

#w2v_model_novice.init_sims(replace = True)
#w2v_model_exp.init_sims(replace = True)

#w2v_model_novice.save("results/word2vec_novice_300.model")
#w2v_model_exp.save("results/word2vec_exp_300.model")

INFO - 17:17:56: precomputing L2-norms of word weight vectors
INFO - 17:17:56: saving Word2Vec object under results/word2vec_exp_300.model, separately None
INFO - 17:17:56: storing np array 'vectors' to results/word2vec_exp_300.model.wv.vectors.npy
INFO - 17:17:59: not storing attribute vectors_norm
INFO - 17:17:59: storing np array 'syn1neg' to results/word2vec_exp_300.model.trainables.syn1neg.npy
INFO - 17:18:02: not storing attribute cum_table
INFO - 17:18:02: saved results/word2vec_exp_300.model


In [109]:
#w2v_model.wv.most_similar(positive=["macron"])
#w2v_model.wv.most_similar(negative=["promesse"])
#w2v_model.wv.similarity("élection", 'présidentielle')
#w2v_model.wv.similarity("sport", 'études')
#print(w2v_model.wv.similarity("macron", 'droite'))
#print(w2v_model.wv.similarity("macron", 'gauche'))
#w2v_model.wv.doesnt_match(['gauche', 'président', 'droite'])
#w2v_model.wv.most_similar(positive=["père", "femme"], negative = ['homme'], topn=3)

## Construction du modèle des corpus regroupés 

In [4]:
df_all = df2.copy().dropna()

INFO - 22:46:18: NumExpr defaulting to 4 threads.


In [5]:
def ajoute_origine(liste, row):
    
    L = [i + '_' + row.Groupe for i in liste]
    
    return L

In [7]:
sent = [ajoute_origine(row.tokenized_replique.split(), row) for _, row in df_all.iterrows()]

In [9]:
phrases = Phrases(sent, min_count = 3, progress_per = 20000)

INFO - 22:52:40: collecting all words and their counts
INFO - 22:52:40: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 22:52:40: collected 30132 word types from a corpus of 25360 words (unigram + bigrams) and 1000 sentences
INFO - 22:52:40: using 30132 counts as vocab in Phrases<0 vocab, min_count=3, threshold=10.0, max_vocab_size=40000000>


In [10]:
bigram = Phraser(phrases)

INFO - 22:52:44: source_vocab length 30132
INFO - 22:52:45: Phraser built with 164 phrasegrams


In [11]:
sentences = bigram[sent]

In [42]:
word_freq = defaultdict(int)

for sent in sentences:
    for i in sent:
        word_freq[i] += 1

In [43]:
model_all = Word2Vec(
                     window = 4,
                     size = 300,
                     sample = 6e-5, 
                     alpha = 0.03, 
                     min_alpha = 0.0007, 
                     negative = 20,
                     workers = multiprocessing.cpu_count() - 1,
                     compute_loss = True)

In [44]:
t = time()

model_all.build_vocab(sentences, progress_per = 10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 00:02:35: collecting all words and their counts
INFO - 00:02:35: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:02:36: PROGRESS: at sentence #10000, processed 229546 words, keeping 50408 word types
INFO - 00:02:36: PROGRESS: at sentence #20000, processed 432262 words, keeping 73268 word types
INFO - 00:02:37: PROGRESS: at sentence #30000, processed 647821 words, keeping 91182 word types
INFO - 00:02:38: PROGRESS: at sentence #40000, processed 869439 words, keeping 105781 word types
INFO - 00:02:39: PROGRESS: at sentence #50000, processed 1074191 words, keeping 115247 word types
INFO - 00:02:40: PROGRESS: at sentence #60000, processed 1253985 words, keeping 123554 word types
INFO - 00:02:41: PROGRESS: at sentence #70000, processed 1451066 words, keeping 132583 word types
INFO - 00:02:41: PROGRESS: at sentence #80000, processed 1623164 words, keeping 137810 word types
INFO - 00:02:42: PROGRESS: at sentence #90000, processed 1842888 words, keeping 14499

Time to build vocab: 1.27 mins


INFO - 00:03:52: EPOCH 1 - PROGRESS: at 0.66% examples, 59610 words/s, in_qsize 0, out_qsize 0
INFO - 00:03:53: EPOCH 1 - PROGRESS: at 2.01% examples, 85645 words/s, in_qsize 0, out_qsize 0
INFO - 00:03:54: EPOCH 1 - PROGRESS: at 3.54% examples, 97725 words/s, in_qsize 0, out_qsize 0
INFO - 00:03:55: EPOCH 1 - PROGRESS: at 5.22% examples, 105032 words/s, in_qsize 0, out_qsize 0
INFO - 00:03:56: EPOCH 1 - PROGRESS: at 6.71% examples, 107565 words/s, in_qsize 0, out_qsize 0
INFO - 00:03:57: EPOCH 1 - PROGRESS: at 8.11% examples, 109629 words/s, in_qsize 0, out_qsize 0
INFO - 00:03:58: EPOCH 1 - PROGRESS: at 9.86% examples, 112342 words/s, in_qsize 0, out_qsize 0
INFO - 00:03:59: EPOCH 1 - PROGRESS: at 11.44% examples, 110798 words/s, in_qsize 0, out_qsize 0
INFO - 00:04:00: EPOCH 1 - PROGRESS: at 12.97% examples, 111225 words/s, in_qsize 0, out_qsize 0
INFO - 00:04:01: EPOCH 1 - PROGRESS: at 14.51% examples, 111292 words/s, in_qsize 0, out_qsize 0
INFO - 00:04:03: EPOCH 1 - PROGRESS: at 

KeyboardInterrupt: 

In [45]:
t = time()

model_all.train(sentences, total_examples = model_all.corpus_count, epochs = 300, report_delay = 60)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

WARNING - 00:04:49: Effective 'alpha' higher than previous training cycles
INFO - 00:04:49: training model with 3 workers on 127983 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=4
INFO - 00:04:50: EPOCH 1 - PROGRESS: at 1.15% examples, 107864 words/s, in_qsize 0, out_qsize 0
INFO - 00:05:50: EPOCH 1 - PROGRESS: at 93.63% examples, 108384 words/s, in_qsize 0, out_qsize 0
INFO - 00:05:54: worker thread finished; awaiting finish of 2 more threads
INFO - 00:05:54: worker thread finished; awaiting finish of 1 more threads
INFO - 00:05:54: worker thread finished; awaiting finish of 0 more threads
INFO - 00:05:54: EPOCH - 1 : training on 9442094 raw words (7073918 effective words) took 64.8s, 109178 effective words/s
INFO - 00:05:55: EPOCH 2 - PROGRESS: at 1.06% examples, 102608 words/s, in_qsize 0, out_qsize 0
INFO - 00:06:55: EPOCH 2 - PROGRESS: at 96.40% examples, 111655 words/s, in_qsize 0, out_qsize 0
INFO - 00:06:57: worker thread finished; awaiting finish

INFO - 00:22:49: EPOCH 16 - PROGRESS: at 0.58% examples, 50618 words/s, in_qsize 0, out_qsize 0
INFO - 00:23:49: EPOCH 16 - PROGRESS: at 90.40% examples, 105042 words/s, in_qsize 0, out_qsize 0
INFO - 00:23:57: worker thread finished; awaiting finish of 2 more threads
INFO - 00:23:57: worker thread finished; awaiting finish of 1 more threads
INFO - 00:23:57: worker thread finished; awaiting finish of 0 more threads
INFO - 00:23:57: EPOCH - 16 : training on 9442094 raw words (7073789 effective words) took 69.2s, 102224 effective words/s
INFO - 00:23:58: EPOCH 17 - PROGRESS: at 0.83% examples, 75743 words/s, in_qsize 0, out_qsize 0
INFO - 00:24:58: EPOCH 17 - PROGRESS: at 78.21% examples, 92909 words/s, in_qsize 0, out_qsize 0
INFO - 00:25:13: worker thread finished; awaiting finish of 2 more threads
INFO - 00:25:13: worker thread finished; awaiting finish of 1 more threads
INFO - 00:25:13: worker thread finished; awaiting finish of 0 more threads
INFO - 00:25:13: EPOCH - 17 : training o

INFO - 00:41:13: worker thread finished; awaiting finish of 2 more threads
INFO - 00:41:13: worker thread finished; awaiting finish of 1 more threads
INFO - 00:41:13: worker thread finished; awaiting finish of 0 more threads
INFO - 00:41:13: EPOCH - 31 : training on 9442094 raw words (7072828 effective words) took 77.2s, 91617 effective words/s
INFO - 00:41:14: EPOCH 32 - PROGRESS: at 0.99% examples, 93835 words/s, in_qsize 0, out_qsize 0
INFO - 00:42:14: EPOCH 32 - PROGRESS: at 86.92% examples, 100708 words/s, in_qsize 0, out_qsize 0
INFO - 00:42:23: worker thread finished; awaiting finish of 2 more threads
INFO - 00:42:23: worker thread finished; awaiting finish of 1 more threads
INFO - 00:42:23: worker thread finished; awaiting finish of 0 more threads
INFO - 00:42:23: EPOCH - 32 : training on 9442094 raw words (7073074 effective words) took 69.7s, 101429 effective words/s
INFO - 00:42:24: EPOCH 33 - PROGRESS: at 1.15% examples, 107610 words/s, in_qsize 0, out_qsize 0
INFO - 00:43:2

INFO - 00:58:22: EPOCH - 46 : training on 9442094 raw words (7073173 effective words) took 69.1s, 102392 effective words/s
INFO - 00:58:23: EPOCH 47 - PROGRESS: at 1.15% examples, 101219 words/s, in_qsize 0, out_qsize 0
INFO - 00:59:23: EPOCH 47 - PROGRESS: at 95.67% examples, 110672 words/s, in_qsize 1, out_qsize 0
INFO - 00:59:26: worker thread finished; awaiting finish of 2 more threads
INFO - 00:59:26: worker thread finished; awaiting finish of 1 more threads
INFO - 00:59:26: worker thread finished; awaiting finish of 0 more threads
INFO - 00:59:26: EPOCH - 47 : training on 9442094 raw words (7072159 effective words) took 64.2s, 110084 effective words/s
INFO - 00:59:28: EPOCH 48 - PROGRESS: at 0.42% examples, 33904 words/s, in_qsize 0, out_qsize 0
INFO - 01:00:28: EPOCH 48 - PROGRESS: at 98.02% examples, 113430 words/s, in_qsize 0, out_qsize 0
INFO - 01:00:29: worker thread finished; awaiting finish of 2 more threads
INFO - 01:00:29: worker thread finished; awaiting finish of 1 mor

INFO - 03:06:22: EPOCH 64 - PROGRESS: at 1.45% examples, 132055 words/s, in_qsize 0, out_qsize 0
INFO - 03:07:17: worker thread finished; awaiting finish of 2 more threads
INFO - 03:07:17: worker thread finished; awaiting finish of 1 more threads
INFO - 03:07:17: worker thread finished; awaiting finish of 0 more threads
INFO - 03:07:17: EPOCH - 64 : training on 9442094 raw words (7074689 effective words) took 55.9s, 126566 effective words/s
INFO - 03:07:18: EPOCH 65 - PROGRESS: at 1.45% examples, 130707 words/s, in_qsize 0, out_qsize 0
INFO - 03:08:08: worker thread finished; awaiting finish of 2 more threads
INFO - 03:08:08: worker thread finished; awaiting finish of 1 more threads
INFO - 03:08:08: worker thread finished; awaiting finish of 0 more threads
INFO - 03:08:08: EPOCH - 65 : training on 9442094 raw words (7072892 effective words) took 51.3s, 137869 effective words/s
INFO - 03:08:09: EPOCH 66 - PROGRESS: at 1.45% examples, 131904 words/s, in_qsize 0, out_qsize 0
INFO - 03:09:

...

INFO - 06:16:45: worker thread finished; awaiting finish of 0 more threads
INFO - 06:16:45: EPOCH - 251 : training on 9442094 raw words (7075463 effective words) took 64.0s, 110558 effective words/s
INFO - 06:16:46: EPOCH 252 - PROGRESS: at 1.06% examples, 100434 words/s, in_qsize 0, out_qsize 0
INFO - 06:17:46: EPOCH 252 - PROGRESS: at 95.77% examples, 110937 words/s, in_qsize 0, out_qsize 0
INFO - 06:17:48: worker thread finished; awaiting finish of 2 more threads
INFO - 06:17:48: worker thread finished; awaiting finish of 1 more threads
INFO - 06:17:48: worker thread finished; awaiting finish of 0 more threads
INFO - 06:17:48: EPOCH - 252 : training on 9442094 raw words (7073806 effective words) took 63.7s, 110996 effective words/s
INFO - 06:17:50: EPOCH 253 - PROGRESS: at 1.15% examples, 103129 words/s, in_qsize 0, out_qsize 0
INFO - 06:18:50: EPOCH 253 - PROGRESS: at 94.95% examples, 109872 words/s, in_qsize 0, out_qsize 0
INFO - 06:18:53: worker thread finished; awaiting finish o

INFO - 06:32:51: EPOCH - 266 : training on 9442094 raw words (7073316 effective words) took 64.2s, 110176 effective words/s
INFO - 06:32:52: EPOCH 267 - PROGRESS: at 1.06% examples, 98265 words/s, in_qsize 0, out_qsize 0
INFO - 06:33:52: EPOCH 267 - PROGRESS: at 94.05% examples, 108891 words/s, in_qsize 0, out_qsize 0
INFO - 06:33:55: worker thread finished; awaiting finish of 2 more threads
INFO - 06:33:56: worker thread finished; awaiting finish of 1 more threads
INFO - 06:33:56: worker thread finished; awaiting finish of 0 more threads
INFO - 06:33:56: EPOCH - 267 : training on 9442094 raw words (7072137 effective words) took 64.9s, 109020 effective words/s
INFO - 06:33:57: EPOCH 268 - PROGRESS: at 1.06% examples, 101572 words/s, in_qsize 0, out_qsize 0
INFO - 06:34:57: EPOCH 268 - PROGRESS: at 94.24% examples, 109158 words/s, in_qsize 0, out_qsize 0
INFO - 06:35:00: worker thread finished; awaiting finish of 2 more threads
INFO - 06:35:00: worker thread finished; awaiting finish of

INFO - 06:49:05: EPOCH 282 - PROGRESS: at 1.06% examples, 98571 words/s, in_qsize 0, out_qsize 0
INFO - 06:50:05: EPOCH 282 - PROGRESS: at 94.05% examples, 108852 words/s, in_qsize 0, out_qsize 0
INFO - 06:50:09: worker thread finished; awaiting finish of 2 more threads
INFO - 06:50:09: worker thread finished; awaiting finish of 1 more threads
INFO - 06:50:09: worker thread finished; awaiting finish of 0 more threads
INFO - 06:50:09: EPOCH - 282 : training on 9442094 raw words (7072268 effective words) took 64.9s, 109000 effective words/s
INFO - 06:50:10: EPOCH 283 - PROGRESS: at 1.06% examples, 101035 words/s, in_qsize 0, out_qsize 0
INFO - 06:51:10: EPOCH 283 - PROGRESS: at 92.80% examples, 107447 words/s, in_qsize 0, out_qsize 0
INFO - 06:51:15: worker thread finished; awaiting finish of 2 more threads
INFO - 06:51:15: worker thread finished; awaiting finish of 1 more threads
INFO - 06:51:15: worker thread finished; awaiting finish of 0 more threads
INFO - 06:51:15: EPOCH - 283 : tr

INFO - 07:06:21: EPOCH 297 - PROGRESS: at 94.05% examples, 108800 words/s, in_qsize 0, out_qsize 0
INFO - 07:06:25: worker thread finished; awaiting finish of 2 more threads
INFO - 07:06:25: worker thread finished; awaiting finish of 1 more threads
INFO - 07:06:25: worker thread finished; awaiting finish of 0 more threads
INFO - 07:06:25: EPOCH - 297 : training on 9442094 raw words (7073415 effective words) took 65.1s, 108738 effective words/s
INFO - 07:06:26: EPOCH 298 - PROGRESS: at 1.06% examples, 102520 words/s, in_qsize 0, out_qsize 0
INFO - 07:07:26: EPOCH 298 - PROGRESS: at 93.44% examples, 108134 words/s, in_qsize 0, out_qsize 0
INFO - 07:07:30: worker thread finished; awaiting finish of 2 more threads
INFO - 07:07:30: worker thread finished; awaiting finish of 1 more threads
INFO - 07:07:30: worker thread finished; awaiting finish of 0 more threads
INFO - 07:07:30: EPOCH - 298 : training on 9442094 raw words (7072397 effective words) took 65.3s, 108292 effective words/s
INFO -

Time to train the model: 424.86 mins


# Sauvegarde du modèle

In [47]:

model_all.init_sims(replace = True)

model_all.save("results/word2vec_all_300.model")

INFO - 07:09:43: precomputing L2-norms of word weight vectors
INFO - 07:09:43: saving Word2Vec object under results/word2vec_all_300.model, separately None
INFO - 07:09:43: storing np array 'vectors' to results/word2vec_all_300.model.wv.vectors.npy
INFO - 07:09:43: not storing attribute vectors_norm
INFO - 07:09:43: storing np array 'syn1neg' to results/word2vec_all_300.model.trainables.syn1neg.npy
INFO - 07:09:44: not storing attribute cum_table
INFO - 07:09:44: saved results/word2vec_all_300.model
